In [ ]:
import pandas_datareader.data as web
import pandas as pd
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
from stock import Stock


In [ ]:
start = datetime.datetime(1928, 1, 1)
end = datetime.date.today()
spyData = Stock('^GSPC', start, end)
spyData.stockData = spyData.stockData.iloc[250:]

spyData.stockData.head()


,Close,Volume,MA20,MA60,MA120,MA250,EMA20,EMA60,EMA120,EMA250,MAScore,PriceScore,SumScore,Chg,%Chg
Date,,,,,,,,,,,,,,,
1929-01-02,24.81,0,23.1925,22.684667,21.405583,19.96540,23.444937,22.695886,21.731696,19.976027,100.0,100.0,100.0,0.46,1.89%
1929-01-03,24.86,0,23.2400,22.744667,21.454833,19.99396,23.579705,22.766841,21.783404,20.014943,100.0,100.0,100.0,0.05,0.2%
1929-01-04,24.85,0,23.2920,22.805167,21.505167,20.02316,23.700686,22.835141,21.834091,20.053470,100.0,100.0,100.0,-0.01,-0.04%
1929-01-07,24.25,0,23.3590,22.857500,21.549250,20.04952,23.753001,22.881530,21.874024,20.086908,100.0,100.0,100.0,-0.6,-2.41%
1929-01-08,24.17,0,23.4495,22.907333,21.594833,20.07620,23.792715,22.923775,21.911974,20.119443,100.0,100.0,100.0,-0.08,-0.33%


In [ ]:
class Period:
    def __init__(self):
        self.startIndex = None
        self.startDate = None
        self.startPrice = None
        self.endIndex = None
        self.endDate = None
        self.endPrice = None
        self.periodDays = None
        self.priceDiff = None

    def __str__(self):
        return "Bear period from {startIndex}-----{startDate}--{startPrice} $ to {endIndex}----{endDate}--{endPrice} $, period is {period} days and lost is {priceDiff} %".format(startIndex=self.startIndex, startDate=str(self.startDate)[0:10], startPrice=self.startPrice, endIndex=self.endIndex, endDate=str(self.endDate)[0:10], endPrice=self.endPrice, period=self.periodDays, priceDiff=self.priceDiff)

    def to_dis(self):
        return {
            'startIndex': self.startIndex,
            'startDate': self.startDate,
            'startPrice': self.startPrice,
            'endIndex': self.endIndex,
            'endDate': self.endDate,
            'endPrice': self.endPrice,
            'periodDays': self.periodDays,
            'priceDiff': self.priceDiff
        }


class ScoreAnalysis:
    def __init__(self, stock, priceScoreWeight=0.5, threshold=50) -> None:
        self.bearPeriod = []
        self.priceScoreWeight = priceScoreWeight
        self.threshold = threshold
        self.stock = stock.updateSumScore(priceScoreWeight)
        self.bearPeriodDF = None
        self.lostSum = None
        self.record = []
        self.earningRate = 0
        self.lostSave = 0

    def __str__(self) -> str:
        return "ScoreAnalysis->priceScoreWeight:{priceScoreWeight}, threshold:{threshold}, earningRate:{earningRate},lostSave:{lostSave}".format(priceScoreWeight=self.priceScoreWeight, threshold=self.threshold, earningRate=self.earningRate, lostSave=self.lostSave)

    def getBearPeriod(self):
        isbearPeriod = False
        for i in range(len(self.stock.stockData)):
            if not isbearPeriod:
                if self.stock.stockData.iloc[i]['SumScore'] < self.threshold:
                    isbearPeriod = True
                    newPeriod = Period()
                    newPeriod.startIndex = i
                    newPeriod.startDate = self.stock.stockData.index[i]
                    newPeriod.startPrice = self.stock.stockData.iloc[i]['Close']
                    self.bearPeriod.append(newPeriod)
                    continue
                else:
                    continue
            else:
                if self.stock.stockData.iloc[i]['SumScore'] >= self.threshold:
                    isbearPeriod = False
                    period = self.bearPeriod[-1]
                    period.endIndex = i
                    period.endDate = self.stock.stockData.index[i]
                    period.endPrice = self.stock.stockData.iloc[i]['Close']
                    endDate = datetime.datetime.strptime(
                        period.endDate, '%Y-%m-%d')
                    startDate = datetime.datetime.strptime(
                        period.startDate, '%Y-%m-%d')
                    period.periodDays = (
                        endDate - startDate).days
                    period.priceDiff = round(
                        (period.endPrice - period.startPrice)*100/period.startPrice, 2)
                    self.lostSave = self.lostSave+period.priceDiff
                    continue
                else:
                    continue

    def caculateIncome(self):
        initAmount = 10000
        shares = math.floor(
            initAmount/self.stock.stockData.iloc[0]['Close'])
        shareValue = shares * self.stock.stockData.iloc[0]['Close']
        initAmount = shareValue
        self.record.append({'Date': self.stock.stockData.index[0], 'action': 'buy', 'price': self.stock.stockData.iloc[
            0], 'shares': shares, 'shareValue': shareValue, 'earning': 0, 'earningRate': 0})
        i = 0
        while i < len(self.bearPeriod):
            period = self.bearPeriod[i]
            shareValue = shares*period.startPrice
            earning = shareValue-initAmount
            earningRate = earning*100/initAmount
            self.record.append({'Date': period.startDate, 'action': 'sell', 'price': period.startPrice,
                                'shares': shares, 'shareValue': shareValue, 'earning': earning, 'earningRate': earningRate})
            shares = math.floor(shareValue/period.endPrice)
            self.record.append({'Date': period.endDate, 'action': 'buy', 'price': period.endPrice,
                                'shares': shares, 'shareValue': shareValue, 'earning': earning, 'earningRate': earningRate})
            i = i+1
        shareValue = shares*self.stock.stockData.iloc[-1]['Close']
        earning = shareValue-initAmount
        earningRate = earning*100/initAmount
        self.record.append({'Date': self.stock.stockData.index[-1], 'action': 'sell', 'price': self.stock.stockData.iloc[-1]['Close'],
                            'shares': shares, 'shareValue': shareValue, 'earning': earning, 'earningRate': earningRate})
        self.recordDf = pd.DataFrame.from_records(
            record for record in self.record)
        self.earningRate = earningRate


In [ ]:
scoreAnalysis = ScoreAnalysis(spyData)
print(scoreAnalysis)
scoreAnalysis.getBearPeriod()
print(scoreAnalysis)


In [ ]:
def analysisMaximunBenefit(stock):
    bestAnalysis = ScoreAnalysis(stock)
    bestAnalysis.getBearPeriod()
    bestAnalysis.caculateIncome()
    priceScoreWeight = 0
    threshold = 0
    while priceScoreWeight <= 1:
        threshold = 0
        while threshold <= 100:
            scoreAnalysis = ScoreAnalysis(stock, priceScoreWeight, threshold)
            scoreAnalysis.getBearPeriod()
            scoreAnalysis.caculateIncome()
            # print("analysis {priceScoreWeight},{threshold},{benefitRate}".format(priceScoreWeight=priceScoreWeight,threshold= threshold,benefitRate=scoreAnalysis.earningRate))
            if scoreAnalysis.earningRate > bestAnalysis.earningRate:
                bestAnalysis = scoreAnalysis
            threshold = threshold+5
        priceScoreWeight = priceScoreWeight+0.1
    print(bestAnalysis)


def analysisMaximunLostSave(stock):
    bestAnalysis = ScoreAnalysis(stock)
    bestAnalysis.getBearPeriod()
    priceScoreWeight = 0
    threshold = 0
    while priceScoreWeight <= 1:
        threshold = 0
        while threshold <= 100:
            scoreAnalysis = ScoreAnalysis(stock, priceScoreWeight, threshold)
            scoreAnalysis.getBearPeriod()
            print("analysis {priceScoreWeight},{threshold},{lostSave}".format(
                priceScoreWeight=priceScoreWeight, threshold=threshold, lostSave=scoreAnalysis.lostSave))
            if scoreAnalysis.lostSave < bestAnalysis.lostSave:
                bestAnalysis = scoreAnalysis
            threshold = threshold+5
        priceScoreWeight = priceScoreWeight+0.1
    print(bestAnalysis)


In [ ]:
# analysisMaximunLostSave(spyData)


金本位废除 1972
ScoreAnalysis->priceScoreWeight:0.7, threshold:15, earningRate:0,lostSave:-110.56000000000002

In [ ]:
ts='1973-01-02'
index=np.where(spyData.stockData.index == ts)
print("金本位废除："+ str(index[0][0]))
print('Optimise: ScoreAnalysis->priceScoreWeight:0, threshold:30, earningRate:37.30,lostSave:1.4100000000000037')
print('Buy and hold: 36.54')


金本位废除：11003
Optimise: ScoreAnalysis->priceScoreWeight:0, threshold:30, earningRate:3730.471305878426,lostSave:1.4100000000000037
Buy and hold: 36.54


In [ ]:
stock=Stock()
stock.label='sp500'
stock.stockData=spyData.stockData.iloc[11003:]
analysisMaximunBenefit(stock)
stock.stockData.iloc[-1]['Close']/stock.stockData.iloc[0]['Close']

/Users/helloworld/App/jupyter/stock.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pricesDF['SumScore'] = score


ScoreAnalysis->priceScoreWeight:0, threshold:30, earningRate:3730.471305878426,lostSave:1.4100000000000037


36.54357682619648

In [ ]:
fig, axes = plt.subplots(19, 1, figsize=(20, 100))
for i in range(19):
    data = spyData.stockData.iloc[i*1250:(i+1)*1250]
    axes[i].plot(data.index, data['Close'], label='Close', color='blue')
    axes[i].twinx()
    plt.plot(data.index, data['SumScore'], label='Score', color='orange')
    axes[i].legend()
    plt.hlines(y=50, xmin=data.index[0], xmax=data.index[-1], colors='red')
    plt.title("SP500 " + str(data.index[0])[0:10] +
              "------"+str(data.index[-1])[0:10], fontsize='large')
    startDate = data.index[0]
    endDate = data.index[-1]
    for period in finalAnalysis.bearPeriod:
        if period.startDate > startDate and period.startDate < endDate:
            plt.vlines(x=period.startDate, ymin=0, ymax=100, color='black')
        if period.endDate > startDate and period.endDate < endDate:
            plt.vlines(x=period.endDate, ymin=0, ymax=100, color='grey')


In [ ]:

increaseRate = 0.02
amount = 5000
Index = 0
cycle = 20
sharePrice = spyData.stockData.values[Index][0]
shares = round(amount/sharePrice)
shareValue = shares*sharePrice
ininData = [[spyData.stockData.index[Index], shareValue, 0, Index, sharePrice, 0,
             shares, shares, shareValue, shareValue, shareValue, sharePrice, 0, 0]]
dfValueAverage = pd.DataFrame(ininData, columns=['Date', 'TargetShareValue', 'ShareValueBeforePurchase', 'PriceIndex', 'SharePrice',  'newEarning', 'DeltaShare', 'Shares',
                                                 'PurchaseValue', 'ShareValue', 'Cost', 'ShareCost', 'Earning', 'EarningRate'])
index = dfValueAverage.iloc[0]['PriceIndex']


In [ ]:
dfValueAverage


In [ ]:
i = 1
while Index < len(spyData.stockData)-cycle:
    previousData = dfValueAverage.iloc[i-1]
    Index = previousData['PriceIndex']+cycle
    currentSharePrice = spyData.stockData.values[Index][0]
    currentShareValueBeforePurchase = currentSharePrice*previousData['Shares']
    newEarning = currentShareValueBeforePurchase-previousData['ShareValue']
    targetShareValue = previousData['ShareValue']*(1+increaseRate)
    deltaShare = math.ceil(
        (targetShareValue-currentShareValueBeforePurchase)/currentSharePrice)
    currentShares = previousData['Shares']+deltaShare
    purchaceValue = deltaShare*currentSharePrice
    currentCost = previousData['Cost']+purchaceValue
    currentShareValue = currentShares*currentSharePrice
    currentEarning = currentShareValue-currentCost
    currentEarningRate = currentEarning*100/currentCost
    shareCost = currentCost/currentShares
    newData = [spyData.stockData.index[Index], targetShareValue, currentShareValueBeforePurchase, Index, currentSharePrice, newEarning, deltaShare, currentShares,
               purchaceValue, currentShareValue, currentCost, shareCost, currentEarning, currentEarningRate]
    dfValueAverage.loc[i] = newData
    i = i+1


In [ ]:
plt.figure(figsize=(20, 20))
plt.plot(dfValueAverage['Date'], dfValueAverage['SharePrice'])
plt.plot(dfValueAverage['Date'], dfValueAverage['ShareCost'])


In [ ]:
increaseAmount = 600
Index = 0
cycle = 20
sharePrice = spyData.stockData.values[Index][0]
shares = math.ceil(increaseAmount/sharePrice)
shareValue = shares*sharePrice
ininData = [[spyData.stockData.index[Index],  0, Index, sharePrice, 0,
             shares, shares, shareValue, shareValue, shareValue, sharePrice, 0, 0]]
dfCurrencyAverage = pd.DataFrame(ininData, columns=['Date',  'ShareValueBeforePurchase', 'PriceIndex', 'SharePrice',  'newEarning', 'DeltaShare', 'Shares',
                                                    'PurchaseValue', 'ShareValue', 'Cost', 'ShareCost', 'Earning', 'EarningRate'])
index = dfCurrencyAverage.iloc[0]['PriceIndex']
dfCurrencyAverage


In [ ]:
i = 1
while Index < len(spyData.stockData)-cycle:
    previousData = dfCurrencyAverage.iloc[i-1]
    Index = previousData['PriceIndex']+cycle
    currentSharePrice = spyData.stockData.values[Index][0]
    currentShareValueBeforePurchase = currentSharePrice*previousData['Shares']
    newEarning = currentShareValueBeforePurchase-previousData['ShareValue']
    deltaShare = math.ceil(increaseAmount/currentSharePrice)
    currentShares = previousData['Shares']+deltaShare
    purchaceValue = deltaShare*currentSharePrice
    currentCost = previousData['Cost']+purchaceValue
    currentShareValue = currentShares*currentSharePrice
    currentEarning = currentShareValue-currentCost
    currentEarningRate = currentEarning*100/currentCost
    shareCost = currentCost/currentShares
    newData = [spyData.stockData.index[Index],  currentShareValueBeforePurchase, Index, currentSharePrice, newEarning, deltaShare, currentShares,
               purchaceValue, currentShareValue, currentCost, shareCost, currentEarning, currentEarningRate]
    dfCurrencyAverage.loc[i] = newData
    i = i+1

dfCurrencyAverage


In [ ]:

plt.figure(figsize=(20, 20))
plt.plot(dfCurrencyAverage['Date'], dfCurrencyAverage['SharePrice'])
plt.plot(dfCurrencyAverage['Date'], dfCurrencyAverage['ShareCost'])
